In [1]:
# https://strawberryfields.ai/photonics/demos/run_gate_visualization.html

import time

import strawberryfields as sf
from strawberryfields.ops import *

import numpy as np
from numpy import pi, sqrt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
# set the random seed
np.random.seed(42)

prog = sf.Program(3)

alpha = 0.0
r = np.abs(alpha)
phi = np.angle(alpha)

with prog.context as q:
    # prepare initial states
    Coherent(r, phi) | q[0]
    Squeezed(-2) | q[1]
    Squeezed(2) | q[2]

    # apply gates
    BS = BSgate(pi/4, pi)
    BS | (q[1], q[2])
    BS | (q[0], q[1])

    # Perform homodyne measurements
    MeasureX | q[0]
    MeasureP | q[1]

    # Displacement gates conditioned on
    # the measurements
    Xgate(sqrt(2) * q[0].par) | q[2]
    Zgate(sqrt(2) * q[1].par) | q[2]

eng = sf.Engine('fock', backend_options={"cutoff_dim": 11})


result = eng.run(prog, shots=1, modes=None, compile_options={})
#print(result.samples)
rho2 = np.einsum('kkllij->ij', result.state.dm())